<a href="https://colab.research.google.com/github/selamhabtewold/firstrepo/blob/main/Logistic_Regression_on_Credit_Risk_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Building  a model to classifying the credit risk for a loan applicant.

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Infosys ML Course/credit_risk.csv')
df.head()


Mounted at /content/drive


,over_draft,credit_usage,credit_history,purpose,current_balance,Average_Credit_Balance,employment,location,personal_status,other_parties,...,property_magnitude,cc_age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,<0,6,critical/other existing credit,radio/tv,1169,no known savings,>=7,4,male single,none,...,real estate,67,none,own,2,skilled,1,yes,yes,good
1,0<=X<200,48,existing paid,radio/tv,5951,<100,1<=X<4,2,female div/dep/mar,none,...,real estate,22,none,own,1,skilled,1,none,yes,bad
2,no checking,12,critical/other existing credit,education,2096,<100,4<=X<7,2,male single,none,...,real estate,49,none,own,1,unskilled resident,2,none,yes,good
3,<0,42,existing paid,furniture/equipment,7882,<100,4<=X<7,2,male single,guarantor,...,life insurance,45,none,for free,1,skilled,2,none,yes,good
4,<0,24,delayed previously,new car,4870,<100,1<=X<4,3,male single,none,...,no known property,53,none,for free,2,skilled,2,none,yes,bad


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   over_draft              1000 non-null   object
 1   credit_usage            1000 non-null   int64 
 2   credit_history          1000 non-null   object
 3   purpose                 1000 non-null   object
 4   current_balance         1000 non-null   int64 
 5   Average_Credit_Balance  1000 non-null   object
 6   employment              1000 non-null   object
 7   location                1000 non-null   int64 
 8   personal_status         1000 non-null   object
 9   other_parties           1000 non-null   object
 10  residence_since         1000 non-null   int64 
 11  property_magnitude      1000 non-null   object
 12  cc_age                  1000 non-null   int64 
 13  other_payment_plans     1000 non-null   object
 14  housing                 1000 non-null   object
 15  exist

In [ ]:
# target column is the class column

df['class'].unique()

array(['good', 'bad'], dtype=object)

So, the target column 'class' can take two values -- 'good' and 'bad' which states whether the past loan application was a good or bad credit risk.

From the output of info() function it can be seen that there are no Null values in any of the columns (since every column has the same number of non-null values as the number of rows). However, some of the predictors have categorical values (given by the datatype, 'object'), which need to be encoded into numbers. Note that, target column doesn't need encoding when it has categorical values.

In [ ]:
#  let's encode the catagorical columns in the dataset

x = df.columns.drop('class')

# our target class has categorical variables, it's not needed to encode it
# We just leave it aside
y= df['class']

encoded_df = pd.get_dummies(df[x])
encoded_df.shape



(1000, 61)

Splitting Credit Risk data into Training and testing data

In this example, the data is split into training and test datasets in the ratio of 85:15.

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(encoded_df, y, test_size =0.15, random_state =100)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((850, 61), (150, 61), (850,), (150,))

Building a model

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

model.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
print("train accuracy: ",model.score(x_train, y_train))
print("test accuracy: ", model.score(x_test, y_test))

train accuracy:  0.7752941176470588
test accuracy:  0.74


The finding shows that the accuracy on the test data is similar to the training data. Therefore, it can be assumed that the model is not overfitting the training data.

#Measuring Model Performance using Confusion Matrix
Confusion matrix helps in assessing how good a model is by comparing the actual target values with the predicted target values.

Let us see how to generate the Confusion Matrix for a model in sklearn

In [ ]:
y_pred = model.predict(x_train)
test_pred = model.predict(x_test)

from sklearn.metrics import confusion_matrix

cm_train = confusion_matrix(y_train, y_pred)
pd.DataFrame(cm_train, columns = model.classes_, index = model.classes_)

,bad,good
bad,125,132
good,59,534


The rows of a Confusion Matrix represent the actual target values and the columns represent the predicted target values.

In the above matrix for training data, we can observe that the model predicted -

140 actually 'bad' credit risks as 'bad'

117 actually 'bad' credit risks as 'good'

60 actually 'good' credit risks as 'bad'

533 actually 'good' credit risks as 'good'

In [ ]:
cm_test = confusion_matrix(y_test, test_pred)
pd.DataFrame(cm_test, columns = model.classes_, index = model.classes_)

,bad,good
bad,19,24
good,15,92


In [ ]:
# accuracy

train_correct_predictions = cm_train[0,0] + cm_train[1,1]
total_predictions = cm_train.sum()
accuracy = train_correct_predictions/total_predictions
print('train accuracy: ',accuracy)

train accuracy:  0.7752941176470588


In [ ]:
test_correct_predictions = cm_test[0,0] + cm_test[1,1]
total_predictions = cm_test.sum()
accuracy = test_correct_predictions/total_predictions
print('test accuracy: ',accuracy)

test accuracy:  0.74


We can see that the accuracy scores calculated from Confusion Matrices is very close to the ones given by the score() function.

In [ ]:
# # Importing the required function
from sklearn.metrics import classification_report
# # Generating the report and printing the same
print(classification_report(y_test,test_pred))



              precision    recall  f1-score   support

         bad       0.56      0.44      0.49        43
        good       0.79      0.86      0.83       107

    accuracy                           0.74       150
   macro avg       0.68      0.65      0.66       150
weighted avg       0.73      0.74      0.73       150

